# Students Performance
## Model Trainer

In [9]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import (
    r2_score,
    mean_absolute_error,
    mean_squared_error,
    root_mean_squared_error,
)


In [2]:
df = pd.read_csv("../data/students_performance.csv")
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [3]:
X = df.drop(columns=["math_score"])
y = df["math_score"]

In [ ]:
# Splitting data into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [5]:
X_train.shape, y_train.shape

((800, 7), (800,))

In [6]:
X_test.shape, y_test.shape

((200, 7), (200,))

## Data Preprocessing
### Encoding and Scaling

In [7]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numerical_features = X_train.select_dtypes(include="int64").columns
categorical_features = X_train.select_dtypes(include="object").columns

scaler = StandardScaler()
encoder = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", encoder, categorical_features),
        ("StandardScaler", scaler, numerical_features),
    ]
)

In [8]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

## Evaluation Function

In [10]:
def evaluate_model(y_true: pd.Series, y_pred: pd.Series):
    mae = mean_absolute_error(y_true=y_true, y_pred=y_pred)
    mse = mean_squared_error(y_true=y_true, y_pred=y_pred)
    rmse = root_mean_squared_error(y_true=y_true, y_pred=y_pred)
    r2 = r2_score(y_true=y_true, y_pred=y_pred)
    return mae, mse, rmse, r2

## Model Training

In [ ]:
# Creating a model dictionary
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(random_state=42),
    "Ridge": Ridge(random_state=42),
    "K-Neighbors": KNeighborsRegressor(n_jobs=-1),
    "Random Forest": RandomForestRegressor(n_jobs=-1, random_state=42),
    "AdaBoost": AdaBoostRegressor(random_state=42),
    "CatBoost": CatBoostRegressor(random_seed=42, thread_count=-1),
    "XGBoost": XGBRegressor(),
}

model_list = []
scores = []

for name, model in models.items():
    model.fit(X_train_processed, y_train)

    y_pred = model.predict(X_test_processed)
    mae, mse, rmse, r2 = evaluate_model(y_true=y_test, y_pred=y_pred)

    print(name)
    model_list.append(name)

    print("Model Performance")
    print(f"Mean Absolute Error: {mae:.4f}")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"Root Mean Squared Error: {rmse:.4f}")
    print(f"R2 Score: {r2:.4f}")

    scores.append(r2)
    print("=" * 50)
    print("\n")

Linear Regression
Model Performance
Mean Absolute Error: 4.2148
Mean Squared Error: 29.0952
Root Mean Squared Error: 5.3940
R2 Score: 0.8804


Lasso
Model Performance
Mean Absolute Error: 5.1557
Mean Squared Error: 42.4758
Root Mean Squared Error: 6.5173
R2 Score: 0.8254


Ridge
Model Performance
Mean Absolute Error: 4.2111
Mean Squared Error: 29.0566
Root Mean Squared Error: 5.3904
R2 Score: 0.8806


K-Neighbors
Model Performance
Mean Absolute Error: 5.5820
Mean Squared Error: 52.0512
Root Mean Squared Error: 7.2147
R2 Score: 0.7861


Random Forest
Model Performance
Mean Absolute Error: 4.6565
Mean Squared Error: 36.1887
Root Mean Squared Error: 6.0157
R2 Score: 0.8513


AdaBoost
Model Performance
Mean Absolute Error: 4.6155
Mean Squared Error: 35.7164
Root Mean Squared Error: 5.9763
R2 Score: 0.8532


Learning rate set to 0.039525
0:	learn: 14.6211422	total: 425us	remaining: 425ms
1:	learn: 14.2409909	total: 690us	remaining: 345ms
2:	learn: 13.8600148	total: 981us	remaining: 326ms
3:

In [15]:
pd.DataFrame(list(zip(model_list, scores)), columns=["Model Name", "R2 Score"]).sort_values(
    by=["R2 Score"], ascending=False
)

,Model Name,R2 Score
2,Ridge,0.880592
0,Linear Regression,0.880433
5,AdaBoost,0.853223
4,Random Forest,0.851282
6,CatBoost,0.850410
7,XGBoost,0.827797
1,Lasso,0.825446
3,K-Neighbors,0.786095


In [16]:
df_preds = pd.DataFrame({
    "Actual Value": y_test,
    "Predicted Value": y_pred,
    "Difference": y_test - y_pred
})

df_preds

,Actual Value,Predicted Value,Difference
521,91,72.256554,18.743446
737,53,51.827824,1.172176
740,80,79.382545,0.617455
660,74,74.103836,-0.103836
411,84,84.875084,-0.875084
...,...,...,...
408,52,46.441635,5.558365
332,62,60.034035,1.965965
208,74,67.617630,6.382370
613,65,69.601791,-4.601791
